<a href="https://colab.research.google.com/github/durva11chavan/FinDengine_Chatbot/blob/main/FIN_DENGINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intelligent Chatbot

In [ ]:
pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.2 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=77f88ed071f3975eb9a45a29d54baa82292c1c8e84d01d5c4cc701b9928de8df
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24589 sha256=994c4780062d1aa1103bda5eb185eebf303d2a76f20180d82339e47cfb2ac792
  Stored in directory: /root/.cache/pip/wheels/d6/9c/58/ee3ba36897e890f3ad81e9b730791a153fce20caa4a8a474df
Successfully built bs4 parse


In [ ]:
pip install nltk.download('punkt')

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip install nltk.download('punkt')'


In [ ]:
pip nltk.download('wordnet')

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip nltk.download('wordnet')'


In [ ]:
pip nltk.download('omw-1.4')

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip nltk.download('omw-1.4')'


In [ ]:
import requests
import urllib
import nltk
import string
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from requests_html import HTMLSession, HTML
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

article_text = ''
article_sentences = ''
article_words = ''
final_response = ''

def get_links(search):
    query=search.replace(" ", "+")
    url = "https://www.google.com/search?q="+query
    session = HTMLSession()
    res = session.get(url)
    unwanted_links = ('https://www.google.', 'https://google.', 'https://webcache.googleusercontent.', 'http://webcache.googleusercontent.', 'https://policies.google.','https://support.google.','https://maps.google.','https://www.youtube.')
    unwanted_words = ('Dictionary', 'English', "Explicit")
    links = list(res.html.absolute_links)

    for url in links[:]:
        if url.startswith(unwanted_links):
            links.remove(url)

    span=res.html.find('span')
    for i in span:
        if len(i.text)>35 and not (i.text.startswith(unwanted_words)):
            final_response = i.text
            final_response = re.sub(r'\[[0-9]*\]', ' ', final_response)
            final_response = re.sub(r'^https?:\/\/.*[\r\n]*', '', final_response)
            final_response = re.sub(r'\s+', ' ', final_response)
            print(final_response)
            final_response = ''
            break
    # print(links)
    #generate_response(res)
    # for i in range(len(links)):
    #     create_article_text(links[i])

def create_article_text(url):
    global article_text
    global article_sentences
    global article_words

    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    html = urlopen(req)
    htmlParse = BeautifulSoup(html, 'html.parser')
    text = ''
    for para in htmlParse.find_all("p"):
        text += para.text
    text = text.lower()
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    article_text  += text

    article_sentences = nltk.sent_tokenize(article_text)
    article_words = nltk.word_tokenize(article_text)

wnlemmatizer = nltk.stem.WordNetLemmatizer()
def generate_response(res):
    span=res.html.find('span')
    for i in span:
        if len(i.text)>35:
            final_response = i.text
            final_response = re.sub(r'\[[0-9]*\]', ' ', final_response)
            final_response = re.sub(r'^https?:\/\/.*[\r\n]*', '', final_response)
            final_response = re.sub(r'\s+', ' ', final_response)
            print(final_response)
            final_response = ''
            break

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

def genrate_response(user_input):
    response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = response + "I am sorry, I could not understand you"
        return response
    else:
        response = response + article_sentences[similar_sentence_number]
        return response

continue_dialogue = True
print("Hi! I am Fin Dengine.", end="")
print("You can ask me any question. I'll try my best to answer your question!")
while(continue_dialogue == True):
    print("\nYou: ", end="")
    human_text = input()
    human_text = human_text.lower()

    print("\nFin: ", end="")
    get_links(human_text)
    article_text=''

Hi! I am Fin Dengine.You can ask me any question. I'll try my best to answer your question!

You: bye

Fin: directly to the next round of a competition in the absence of an assigned

You: business

Fin: a person's regular occupation, profession, or trade.

You: what is chat gpt

Fin: ChatGPT is a natural language processing tool driven by AI technology that allows you to have human-like conversations and much more with the ...

You: 

KeyboardInterrupt: ignored